In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import language_modelling_seq2seq

In [2]:
# chess

with open("data/pg5614.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
print(len(char2i))
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"
print(len(i2char))

95
95


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 20
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)
source_1 = encoded[:, :length // 2]
print(source_1.shape)
target_1 = torch.cat((torch.ones(encoded.shape[0], 1).to(device).long(), encoded[:, length // 2:]), axis = 1)
print(target_1.shape)

556949
556929
['\ufeffThe Project Gutenbe', 'The Project Gutenber', 'he Project Gutenberg', 'e Project Gutenberg ', ' Project Gutenberg E']



torch.Size([556929, 20])
torch.Size([556929, 10])
torch.Size([556929, 11])


In [8]:
importlib.reload(language_modelling_seq2seq)    
net = language_modelling_seq2seq.Seq2SeqRNN(char2i, i2char, 
                                            encoder_hidden_units = 100, 
                                            decoder_hidden_units = 100)
net.to(device)
net.fit(source_1, target_1, epochs = 5, batch_size = 150, lr = 0.0001, verbose = 3)

Net parameters: 207,195



Epoch:    1, Loss: 2.5550
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' ee  nh  e', ' h  nh  e ', '   ee  e e', '  ne  e e ', ' ne  e e  ']
greedy_search
['          ', '          ', '          ', '          ', '          ']
tensor([-12.1100, -12.3800, -10.8900,  -8.8300,  -7.9900], device='cuda:0')
sample
['oidretrrfh', ' kowohpa e', 'o tfe ie r', ' . 91e3bco', 'k. mciPcrt']
beam_search
[['          ', '          ', '          ', '          ', '          '],
 ['e         ', 'e         ', 'e         ', ' e        ', 'e         '],
 ['      t   ', 'in        ', '      t   ', 'e         ', ' e        '],
 ['       t  ', '      t   ', '       t  ', '      t   ', '      t   '],
 ['        t ', '        t ', '        t ', '       t  ', '       t  ']]
tensor([[-12.1200, -11.8900, -12.0000, -11.8700, -11.7900],
        [-13.6300, -13.5400, -13.5100, -13.2800, -13.3000],
        [-14.4000, -13.5700, -14.2900, -13.3400, -13.3000],
        [-14.4200, -13.98


Epoch:    2, Loss: 1.9298
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e aone  e', 'ehaote  e ', 'etote  e e', ' anh  e e ', 'ene te e a']
greedy_search
[' the the t', 'e the the ', 'e the the ', ' of the th', 'e the the ']
tensor([-11.8500, -11.7600, -11.0900, -12.4200, -11.7400], device='cuda:0')
sample
['onteradn a', 'yn tWhe. a', 'ary, heuns', ' foKtt, ah', 're\nIovedi ']
beam_search
[[' the the t', 'e the the ', 'e the the ', 'e the the ', 'e the the '],
 ['the the th', 'on the the', 'in the the', ' the the t', 'in the the'],
 [' the there', 'in the the', 'on the the', ' the there', 'on the the'],
 [' the the p', 'ing the th', 'ing the th', 'ind the th', 'ing the th'],
 [' the the a', 'e the ther', 'e the ther', ' the the p', 'e the ther']]
tensor([[-11.6800, -11.4100, -11.2100, -11.5900, -11.2500],
        [-11.7700, -11.6200, -11.7700, -11.6900, -11.7400],
        [-12.0800, -11.8000, -11.7800, -12.1000, -11.7700],
        [-12.5000, -12.4


Epoch:    3, Loss: 1.6498
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e oute  e', 'eeoute  e ', 'eaate  e  ', 'eate  e e ', 'ele te   t']
greedy_search
[' the the t', 'ent of the', 'e the the ', 'e the the ', 'ere the th']
tensor([-11.1100, -12.0400,  -9.9100, -11.1900, -13.2800], device='cuda:0')
sample
['is,\n"t at ', ' fiatn)oug', 'ils hof py', 'igimitid y', 'roulite:.\n']
beam_search
[[' the the t', 'on the the', 'on the the', 'on the the', 'on the the'],
 [' the there', 'e the the ', 'end the th', 'ing the th', 'ing the th'],
 ['the the th', 'the the th', 'ing the th', 'on the tha', 'ing of the'],
 [' the the p', 'the the pa', 'on the tha', 'on the thi', 'ing the Kt'],
 ['the the pa', 'e the ther', 'on the thi', 'on the tho', 'ing the pa']]
tensor([[-11.1100, -10.7700, -10.4700, -10.3600, -10.8600],
        [-11.2000, -11.0400, -11.7100, -11.5700, -11.1000],
        [-11.2200, -11.0800, -11.7600, -12.0300, -11.6100],
        [-11.3100, -11.


Epoch:    4, Loss: 1.5114
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
[' e oute  e', 'ehoute  e ', 'eaate  e  ', 'aute  e e ', 'arerte e t']
greedy_search
[' the pawn ', 'ent of the', 'e the pawn', 'and the pa', 'antere the']
tensor([-11.4500, -10.9000, -10.8700, -11.0100, -13.4900], device='cuda:0')
sample
['.ithr:\na t', ' ofore. \no', 'ainHe\nNrog', 'everaplve.', 'awn: 10m6.']
beam_search
[[' the the p', 'ing the th', 'en in the ', 'he the the', 'ing the th'],
 [' the there', 'ing the pa', 'ing the th', 'his the th', 'and the th'],
 [' the the t', 'ing the Kt', 'ing the pa', 'his the pa', 'ing the pa'],
 [' the ther ', 'ing the pe', 'ing the po', 'ing the pa', 'and the pa'],
 [' the the c', 'ing the po', 'ing the pe', 'ing the po', 'and the Kt']]
tensor([[-10.4500, -10.9400,  -9.6900, -10.3200, -10.5000],
        [-10.5400, -11.3100, -10.2700, -10.8400, -10.7600],
        [-10.5400, -11.4400, -10.7600, -11.0800, -10.9600],
        [-11.0400, -12


Epoch:    5, Loss: 1.4236
Y
['ct Gutenbe', 't Gutenber', ' Gutenberg', 'Gutenberg ', 'utenberg E']
forward
['nk outen e', 'e outen en', 'eauter e  ', 'auter ere ', 'arerter  1']
greedy_search
['nt of the ', 'en the paw', 'e the pawn', 'and and an', 'and and an']
tensor([-10.3700, -12.2200, -10.0700, -11.2400, -14.3700], device='cuda:0')
sample
['ins E Dan ', 'anhang her', 'ans squedk', 'Int astine', 'evet\nof Co']
beam_search
[['there the ', 'en the the', 'en the the', 'on the the', 'ing the th'],
 ['nge the pa', 'en the paw', 'en the paw', 'on the paw', 'ing the pa'],
 ['then the p', 'en the can', 'ed the paw', 'and the pa', 'ing there '],
 ['nge the th', 'en the mov', 'en the mov', 'on the can', 'ing the ca'],
 ['nge the po', 'en the pan', 'en the pan', 'on the mov', 'ing the mo']]
tensor([[-11.1400, -11.0100, -10.5800,  -9.9800, -10.4400],
        [-12.0700, -11.8700, -11.3800, -11.1100, -10.8500],
        [-12.2300, -12.3600, -11.7300, -11.1200, -11.1600],
        [-12.2600, -12.4

In [9]:
[net.tensor2text(t) for t in net.beam_search(source_1[:10000], verbose = 1, batch_size = 50)[0]]

[['<START>nge the there the pa',
  '<START>nge the there the th',
  '<START>there the pawns of t',
  '<START>nge the there the po',
  '<START>nge the there the pr'],
 ["<START>ner the Black's Blac",
  '<START>ner the Black there ',
  '<START>ner the Black therer',
  '<START>ner the Black theren',
  '<START>ner the Black there\n'],
 ["<START>ing the King's Black",
  "<START>ing the King's the K",
  "<START>ing the King's the p",
  "<START>ing the King's the B",
  "<START>ing the King's the t"],
 ["<START>ing's Black's Black ",
  "<START>ing's Black's Black\n",
  "<START>ing's Black's Black'",
  "<START>ing's Black's Black,",
  "<START>ing's Black's Blawn "],
 ['<START>and the pawn of the ',
  '<START>or the pawn and the ',
  '<START>and the pawns of the',
  '<START>and the pawn of ther',
  '<START>and the pawn of the\n'],
 ["<START>and the Black's Blac",
  "<START>and the Black's Kt-K",
  "<START>and the Black's Kt-Q",
  "<START>and the Black's Kt-B",
  '<START>and the Black and wi'],
 